In [ ]:
import os
import numpy as np
import pandas as pd
import string
from nltk.corpus import stopwords
import re
import os
from collections import Counter
import json

import ekphrasis
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import gc
import emoji
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import numpy as np
from transformers import BertTokenizer, BertModel, AutoTokenizer, MPNetModel, RobertaModel, BloomModel, OPTModel, AutoModel
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter
from scipy.spatial.distance import jensenshannon
from sklearn.metrics import recall_score
from sklearn.metrics import recall_score
from utils import *

In [ ]:
def print_report(y_pred, y):
    print(classification_report(y, y_pred, digits=3))

<h6>Loading the dataset</h6>

In [ ]:
text_processor = get_text_preprocessor()

In [ ]:
text_train, train_labels = get_data("data/tweet_goEmotions_train.csv")
text_val, val_labels = get_data("data/tweet_goEmotions_val.csv")
text_test, test_labels = get_data("data/tweet_goEmotions_test.csv")

In [ ]:
print(len(text_train))
print(len(text_val))
print(len(text_test))

<h6>Text pre-processing</h6>

In [ ]:
text_train = pre_process_text(text_train, 'slang.txt', text_processor)
print("******************************** Train set Completed ***************************************")
text_val = pre_process_text(text_val, 'slang.txt', text_processor)
print("******************************** Val set Completed ***************************************")
text_test = pre_process_text(text_test, 'slang.txt', text_processor)
print("******************************** Test set Completed ***************************************")

<h8>Longest tokens analysis</h8>

In [ ]:
u = lambda text: len(text.split(" "))
sentence_lengths = []
for x in text_train:
    sentence_lengths.append(u(x))
print(sorted(sentence_lengths)[-100:])
print(max(sentence_lengths))

<h6>Text pre-processing completed</h6>

In [ ]:
# Label size should be changed corresponding to different datasets.
# 28 for goEmotions
# 32 for empatheticDialogue
# 10 for tweetEmotion
# 8  for tweetHate

config = {
    'model': AutoModel.from_pretrained("microsoft/deberta-xlarge"),
    'model_type': "microsoft/deberta-xlarge",
    'embed_dim': 1024,
    'label_size': 28
}

In [ ]:
class datasetLoader(Dataset):

    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        input_ids = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        label = self.labels[idx]
        return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": torch.tensor(label)}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config['model_type'])

In [ ]:
# max_length: goEmotions - 40
# max_length: empathetic - 40
# max_length: tweetEmotion - 50
# max_length: tweetHate - 70

train_encodings = tokenizer(list(text_train), max_length=40, truncation=True, padding='longest', return_tensors='pt')
val_encodings = tokenizer(list(text_val), max_length=40, truncation=True, padding='longest', return_tensors='pt')
test_encodings = tokenizer(list(text_test), max_length=40, truncation=True, padding='longest', return_tensors='pt')

In [ ]:
train_dataset = datasetLoader(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
val_dataset = datasetLoader(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)
test_dataset = datasetLoader(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# Checking the datatype
for dl in train_dataloader:
    print((dl['input_ids'].dtype))
    print((dl['attention_mask'].dtype))
    break

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda:4"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

<h9>Defining the main model class</h9>

In [ ]:
# Classification head is defined seperately to easily resuse with early exiting.
# Config dictionary is used to define the classification head.

class exit_classification_head(nn.Module):
    def __init__(self, embed_dim, label_size):
        super(exit_classification_head, self).__init__()
        self.embed_dim = embed_dim
        self.label_size = label_size

        self.linear_relu_stack = nn.Sequential(
            nn.Linear(self.embed_dim, 128),
            nn.ReLU(),
            nn.Linear(128, self.label_size)
        )

    def forward(self, feature_logits):
        logits = self.linear_relu_stack(feature_logits)
        return logits

In [ ]:
# del model
# del config
# del optimizer
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
class contrastiveNetwork(nn.Module):
    def __init__(self, config):
        super(contrastiveNetwork, self).__init__()

        self.embed_dim = config['embed_dim']
        self.label_size = config['label_size']

        self.model = config['model']
        self.classification_head = exit_classification_head(self.embed_dim, self.label_size)
        self.softmax = nn.Softmax(dim=1)


    def forward(self, input_ids, attention_mask):
        pooler = self.model(input_ids, attention_mask).last_hidden_state[:,0,:]
        logits = self.classification_head(pooler)
        probs = self.softmax(logits)
        return logits, probs


model = contrastiveNetwork(config)
model.to(device)
print(model)

In [ ]:
for param in model.model.embeddings.parameters():
    param.requires_grad = False

In [ ]:
att_layers = [i for i in range(33)]
for j in att_layers:
    for param in model.model.encoder.layer[j].parameters():
        param.requires_grad = False

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=4e-6)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    model.train()
    
    loop = tqdm(enumerate(dataloader), leave=False, total=len(dataloader))

    for batch, dl in loop:
        input_ids = dl['input_ids'].to(device)
        attention_mask = dl['attention_mask'].to(device)
        y = dl['labels'].to(device)

        # Compute prediction error
        pred, probs = model(input_ids, attention_mask)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(y)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
# This function sets the forward hooks which store the output logits.
# Only ([CLS]/<s> token logits are stored for classification). For decoding index of current step can be passed.

def getlogits(name):
    def hook(model, input, output):
        output = output.detach()
        logits[name] = output[:,0,:]
    return hook

# Initialising the hooks on the output layers of each attention block
def initialise_hooks(model):
    hooks = {}
    blocks = len(model.model.encoder.layer)
    for i in range(0, blocks):
        hooks[i] = model.model.encoder.layer[i].output.register_forward_hook(getlogits('block_' + str(i)))
    return hooks

# Removing the hooks once eval is done -- cleanup step
def detach_hooks(hooks, model):
    blocks = len(model.model.encoder.layer)
    for i in range(0, blocks):
        hooks[i].remove()

# Get the softmax probabilities for the outputs from each block ([CLS]/<s> token)
def get_early_logits(early_probs, model, attention_mask):
    for i in range(len(early_probs)):
        early_logits = logits['block_' + str(i)]
        early_logits = model.classification_head(early_logits)
        probs = model.softmax(early_logits)
        early_probs['block_' + str(i)].append(probs.cpu())
    return early_probs

In [ ]:
def evaluation(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    val_loss = 0
    Y_pred = []
    Y_true = []

    # Setting up early exiting parameters.
    global logits
    logits = {}
    early_probs = {}
    hooks = initialise_hooks(model)
    for i in range(0, len(model.model.encoder.layer)):
        early_probs['block_' + str(i)] = []

    # Early exit parameter setup finished
    loop = tqdm(enumerate(dataloader), leave=False, total=len(dataloader))

    with torch.no_grad():
        for batch, dl in loop:
            input_masks = dl['input_ids'].to(device)
            attention_mask = dl['attention_mask'].to(device)
            y = dl['labels'].to(device)
            pred, probs = model(input_masks, attention_mask)
            Y_pred.append(pred.argmax(1).cpu())
            Y_true.append(y.cpu())
            early_probs = get_early_logits(early_probs, model, attention_mask)
    
    detach_hooks(hooks, model)
    # Checking logits output

    for i in range(len(early_probs)):
        early_probs['block_' + str(i)] = torch.cat(early_probs['block_' + str(i)])

    # Whole dataset validation loop
    Y_pred = torch.cat(Y_pred)
    Y_true = torch.cat(Y_true)
    print_report(Y_pred, Y_true)
    return early_probs

In [ ]:
# # Training step
# epochs = 3
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     train(train_dataloader, model, loss_fn, optimizer)
# print("Done!")

In [ ]:
# torch.save(model.state_dict(), 'goEmotions_deberta_xl.pth')
# model.load_state_dict(torch.load('goEmotions_deberta_xl.pth', map_location=device))

In [ ]:
# Dictionary of early exiting probabilities. 
# Stores woth keys block_n where n is the layer number.
early_probs = evaluation(test_dataloader, model, loss_fn)

In [ ]:
# Can be used to print the result from early exiting layers.
# print_report(early_probs['block_46'].argmax(1), test_labels)

<h6>Amateur Layer Selection</h6>

In [ ]:
# Dynamic amateur layer selection.
# This function dynamically selects the 
# n_blocks defines the upper limit of layers not included for amateur layer selection.

n_blocks = 45

def get_max_diff(early_probs):
    final_layer_probs = early_probs['block_47']
    js_data = []
    not_blocks = ["block_" + str(i) for i in range(0,n_blocks)] 
    for block, probs in early_probs.items():
        if(block in not_blocks):
            js_data.append(np.zeros(len(probs)))
            continue
        js_data.append((1-np.array(torch.nn.functional.cosine_similarity(final_layer_probs, probs, dim=1))/2))
    return js_data

In [ ]:
js = np.array(get_max_diff(early_probs))
js = np.argmax(js.T, axis=1)

In [ ]:
Counter(js)

In [ ]:
dynamic_amateur = []
i = 0
for j in js:
    t = 'block_'+str(j)
    dynamic_amateur.append(early_probs[t][i].numpy())
    i+=1
dynamic_amateur = np.array(dynamic_amateur)

In [ ]:
np.shape(dynamic_amateur)

<h5>Applying contrastive decoding</h5>

In [ ]:
probs_base = dynamic_amateur
probs_large = early_probs['block_47'].numpy()

In [ ]:
print(np.shape(probs_base))
print(np.shape(probs_large))

In [ ]:
# Main class for CD.
# The set_hyper_params function is used to set dynamic contrastive strength based on a bool paramter 'dynamic'.
# beta is used for extrapolation layer is dynamic is 'True' else it represents the contrastive strength.

class contrastive_decoding():
    def __init__(self, p_exp_distribution, p_ama_distribution, jsd):
        self.alpha = 1.0
        self.beta = None
        
        self.p_exp = p_exp_distribution
        self.p_ama = p_ama_distribution
        self.jsd = jsd
        
    def initialise_beta(self, beta):
        l_3 = np.ones(len(self.jsd))*beta
        l_2 = np.ones(len(self.jsd))*47.0
        l_1 = self.jsd
        beta_d = (l_3-l_2)/(l_2-l_1)
        beta_d = np.expand_dims(beta_d, axis=1)
        return beta_d

    def set_hyper_params(self, alpha, beta, dynamic=False):
        # Seperate setter function as it makes it easy to evaluate CD for different hyperparameters with a single object.
        self.alpha = alpha
        if not dynamic:
            self.beta = beta
        else:
            self.beta = self.initialise_beta(beta)

    def get_plausibility_constraint(self, p_exp):
        p_exp_max = np.max(p_exp, axis=1)
        p_exp_max = np.expand_dims(p_exp_max, axis=1)
        plausibility_ratio = np.divide(p_exp, p_exp_max)
        plausibility_mask = (plausibility_ratio >= self.alpha).astype(dtype=int)
        return plausibility_mask

    def get_contrastive_pred(self):
        plausibility_mask = self.get_plausibility_constraint(self.p_exp)
        cd_score = (1+self.beta)*np.log(self.p_exp) - (self.beta)*np.log(self.p_ama)
        cd_score = np.multiply(cd_score, plausibility_mask)
        cd_score[cd_score == 0] = -999999999
        return np.argmax(cd_score, axis=1)

    def get_contrastive_prob(self):
        plausibility_mask = self.get_plausibility_constraint(self.p_exp)
        cd_score = (1+self.beta)*np.log(self.p_exp) - (self.beta)*np.log(self.p_ama)
        return cd_score

    def evaluate_cd(self, y):
        pred_cd = self.get_contrastive_pred()
        return f1_score(y, pred_cd, average='macro')

    def evaluate_exp(self, y):
        pred_exp = np.argmax(self.p_exp, axis=-1)
        return f1_score(y, pred_exp, average='macro')

In [ ]:
# This function calls the contrastive decoding class and plots the results.
def plot(cd_, alpha, beta_, true_labels, dynamic = False):
    f1_exp = np.full(len(beta_), cd_.evaluate_exp(true_labels))
    f1_val_cd = np.zeros(len(beta_))

    for i, beta in enumerate(beta_):
        cd_.set_hyper_params(alpha, beta, dynamic)
        f1_val_cd[i] = cd_.evaluate_cd(true_labels)
    
    plt.title("alpha: " + str(alpha))
    plt.plot(beta_, f1_val_cd, marker='o', color='green')
    plt.plot(beta_, f1_exp, color='green', linestyle='dashed')
    plt.show()

In [ ]:
cd_obj = contrastive_decoding(probs_large, probs_base, js)

In [ ]:
cd_obj.set_hyper_params(1.0, 0.0)
print(cd_obj.evaluate_exp(test_labels))
print(cd_obj.evaluate_cd(test_labels))

In [ ]:
# Plots the CD results without dynamic contrastive strength
alpha_ = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
beta_ = np.array([-.25, 0.0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1.0])
for alpha in alpha_:
    plot(cd_obj, alpha, beta_, test_labels, False)

In [ ]:
# Plots the CD results with dynamic contrastive strength. beta_ here contains the extrapolation layer
alpha_ = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
beta_ = np.arange(48,70)
for alpha in alpha_:
    plot(cd_obj, alpha, beta_, test_labels, True)